# Setup
- MTA Ridership beginning in 2025 is from [catalog.data.gov]("https://catalog.data.gov/dataset/mta-subway-hourly-ridership-beginning-2025")
    - CSV format
- MTA Ridership from 2024 is from [data.ny.gov]("https://data.ny.gov/Transportation/MTA-Subway-Hourly-Ridership-2020-2024/wujg-7c2s/about_data")
    - Select rows where <span style="color:orange">transit_timestamp</span> is between `2024-01-01T00:00:00 and 2024-12-31T23:45:00` and <span style="color:orange">borough</span> is `Manhattan`
        - CSV format
- Install <span style="color:lightblue">pandas</span> using `pip install pandas` in terminal if not already installed

In [1]:
!pip install pandas

In [2]:
import pandas as pd

# Cleaning 2025 Subway Ridership

In [3]:
mta_subway_hourly_ridership_2025 = pd.read_csv("data/MTA_Subway_Hourly_Ridership__Beginning_2025.csv")

/var/folders/m1/_k2rjlm94y988l9mx_d_2rhc0000gn/T/ipykernel_21119/3282119758.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  mta_subway_hourly_ridership_2025 = pd.read_csv("data/MTA_Subway_Hourly_Ridership__Beginning_2025.csv")


In [4]:
mta_subway_hourly_ridership_2025.head()

,transit_timestamp,transit_mode,station_complex_id,station_complex,borough,payment_method,fare_class_category,ridership,transfers,latitude,longitude,Georeference
0,01/03/2025 12:00:00 AM,tram,TRAM2,RI Tramway (Roosevelt),Manhattan,omny,OMNY - Full Fare,12,7,40.757340,-73.95412,POINT (-73.95412 40.75734)
1,01/03/2025 06:00:00 AM,tram,TRAM1,RI Tramway (Manhattan),Manhattan,omny,OMNY - Seniors & Disability,1,1,40.761337,-73.96416,POINT (-73.96416 40.761337)
2,01/03/2025 07:00:00 AM,tram,TRAM2,RI Tramway (Roosevelt),Manhattan,metrocard,Metrocard - Unlimited 30-Day,10,0,40.757340,-73.95412,POINT (-73.95412 40.75734)
3,01/03/2025 08:00:00 AM,tram,TRAM1,RI Tramway (Manhattan),Manhattan,metrocard,Metrocard - Unlimited 30-Day,5,0,40.761337,-73.96416,POINT (-73.96416 40.761337)
4,01/03/2025 10:00:00 AM,tram,TRAM1,RI Tramway (Manhattan),Manhattan,omny,OMNY - Students,1,0,40.761337,-73.96416,POINT (-73.96416 40.761337)


In [5]:
# We did not do the borough query for the 2025 data, so we have to do it here:
mta_subway_hourly_ridership_2025 = mta_subway_hourly_ridership_2025[mta_subway_hourly_ridership_2025['borough'] == 'Manhattan']

In [6]:
# We need to filter the data to only include subways as the transit mode
mta_subway_hourly_ridership_2025 = mta_subway_hourly_ridership_2025[mta_subway_hourly_ridership_2025['transit_mode'] == 'subway']

In [7]:
# Removing columns that we don't need in analysis
mta_subway_hourly_ridership_2025 = mta_subway_hourly_ridership_2025.drop(columns=['payment_method','fare_class_category','transit_mode','transfers'])

In [8]:
mta_subway_hourly_ridership_2025.head()
# Looks a lot better!

,transit_timestamp,station_complex_id,station_complex,borough,ridership,latitude,longitude,Georeference
19049,03/09/2025 09:00:00 PM,316,50 St (1),Manhattan,1,40.761726,-73.98385,POINT (-73.98385 40.761726)
19054,01/19/2025 02:00:00 PM,605,"168 St (A,C,1)",Manhattan,3,40.840717,-73.93956,POINT (-73.93956 40.840717)
21861,03/27/2025 02:00:00 AM,118,3 Av (L),Manhattan,2,40.732850,-73.98612,POINT (-73.98612 40.73285)
21866,03/27/2025 10:00:00 AM,160,"72 St (C,B)",Manhattan,242,40.775593,-73.97641,POINT (-73.97641 40.775593)
21875,03/27/2025 07:00:00 PM,405,23 St (6),Manhattan,103,40.739864,-73.98660,POINT (-73.9866 40.739864)


In [9]:
# We need to convert transit_timestamp to a datetime object
mta_subway_hourly_ridership_2025['transit_timestamp'] = pd.to_datetime(mta_subway_hourly_ridership_2025['transit_timestamp'], format='%m/%d/%Y %H:%M:%S %p')

In [10]:
# We should create a separate column that divides the date and time
mta_subway_hourly_ridership_2025['transit_date'] = mta_subway_hourly_ridership_2025['transit_timestamp'].dt.date
mta_subway_hourly_ridership_2025['transit_time'] = mta_subway_hourly_ridership_2025['transit_timestamp'].dt.time

In [11]:
#Now, we need a column that tells us total ridership for each date across all subway stations
mta_subway_hourly_ridership_2025['total_ridership_ondate'] = mta_subway_hourly_ridership_2025.groupby('transit_date')['ridership'].transform('sum')

In [12]:
#Let's also add a column that tells us total ridership for each date and station
mta_subway_hourly_ridership_2025['total_ridership_ondate_and_station'] = mta_subway_hourly_ridership_2025.groupby(['transit_date','station_complex_id'])['ridership'].transform('sum')

In [13]:
#Now, on each date, what is the ridership for each station as a percentage of the total ridership?
mta_subway_hourly_ridership_2025['station_percentage_of_total_ridership'] = mta_subway_hourly_ridership_2025['total_ridership_ondate_and_station'] / mta_subway_hourly_ridership_2025['total_ridership_ondate'] * 100

In [14]:
mta_subway_hourly_ridership_2025.head()
# Looks good

,transit_timestamp,station_complex_id,station_complex,borough,ridership,latitude,longitude,Georeference,transit_date,transit_time,total_ridership_ondate,total_ridership_ondate_and_station,station_percentage_of_total_ridership
19049,2025-03-09 09:00:00,316,50 St (1),Manhattan,1,40.761726,-73.98385,POINT (-73.98385 40.761726),2025-03-09,09:00:00,1129837,1,0.000089
19054,2025-01-19 02:00:00,605,"168 St (A,C,1)",Manhattan,3,40.840717,-73.93956,POINT (-73.93956 40.840717),2025-01-19,02:00:00,1047999,3,0.000286
21861,2025-03-27 02:00:00,118,3 Av (L),Manhattan,2,40.732850,-73.98612,POINT (-73.98612 40.73285),2025-03-27,02:00:00,2379341,4020,0.168954
21866,2025-03-27 10:00:00,160,"72 St (C,B)",Manhattan,242,40.775593,-73.97641,POINT (-73.97641 40.775593),2025-03-27,10:00:00,2379341,5920,0.248808
21875,2025-03-27 07:00:00,405,23 St (6),Manhattan,103,40.739864,-73.98660,POINT (-73.9866 40.739864),2025-03-27,07:00:00,2379341,17875,0.751258


# Cleaning 2024 Subway Ridership

In [15]:
mta_subway_hourly_ridership_2024 = pd.read_csv("data/MTA_Subway_Hourly_Ridership__2024.csv")

In [16]:
mta_subway_hourly_ridership_2024.head()

,transit_timestamp,transit_mode,station_complex_id,station_complex,borough,payment_method,fare_class_category,ridership,transfers,latitude,longitude,Georeference
0,01/01/2024 12:00:00 AM,subway,157,"96 St (C,B)",Manhattan,metrocard,Metrocard - Seniors & Disability,3,0,40.791640,-73.964700,POINT (-73.9647 40.79164)
1,01/01/2024 12:00:00 AM,subway,440,"116 St (2,3)",Manhattan,metrocard,Metrocard - Fair Fare,5,0,40.802097,-73.949620,POINT (-73.94962 40.802097)
2,01/01/2024 12:00:00 AM,subway,404,28 St (6),Manhattan,metrocard,Metrocard - Full Fare,36,0,40.743070,-73.984260,POINT (-73.98426 40.74307)
3,01/01/2024 12:00:00 AM,subway,477,72 St (Q),Manhattan,metrocard,Metrocard - Unlimited 7-Day,44,0,40.768800,-73.958430,POINT (-73.95843 40.7688)
4,01/01/2024 12:00:00 AM,subway,306,125 St (1),Manhattan,omny,OMNY - Seniors & Disability,2,0,40.815582,-73.958374,POINT (-73.958374 40.815582)


In [17]:
# Remove the tram category for transit mode
mta_subway_hourly_ridership_2024 = mta_subway_hourly_ridership_2024[mta_subway_hourly_ridership_2024['transit_mode'] != 'tram']

In [18]:
# Remove columns we don't need for analysis
mta_subway_hourly_ridership_2024 = mta_subway_hourly_ridership_2024.drop(columns=['payment_method','fare_class_category','transit_mode','transfers'])

In [19]:
# We need to convert transit_timestamp to a datetime object
mta_subway_hourly_ridership_2024['transit_timestamp'] = pd.to_datetime(mta_subway_hourly_ridership_2024['transit_timestamp'], format='%m/%d/%Y %H:%M:%S %p')

In [20]:
# We should create a separate column that divides the date and time
mta_subway_hourly_ridership_2024['transit_date'] = mta_subway_hourly_ridership_2024['transit_timestamp'].dt.date
mta_subway_hourly_ridership_2024['transit_time'] = mta_subway_hourly_ridership_2024['transit_timestamp'].dt.time

In [21]:
#Now, we need a column that tells us total ridership for each date across all subway stations
mta_subway_hourly_ridership_2024['total_ridership_ondate'] = mta_subway_hourly_ridership_2024.groupby('transit_date')['ridership'].transform('sum')

In [22]:
#Let's also add a column that tells us total ridership for each date and station
mta_subway_hourly_ridership_2024['total_ridership_ondate_and_station'] = mta_subway_hourly_ridership_2024.groupby(['transit_date','station_complex_id'])['ridership'].transform('sum')

In [23]:
#Now, on each date, what is the ridership for each station as a percentage of the total ridership?
mta_subway_hourly_ridership_2024['station_percentage_of_total_ridership'] = mta_subway_hourly_ridership_2024['total_ridership_ondate_and_station'] / mta_subway_hourly_ridership_2024['total_ridership_ondate'] * 100

In [24]:
mta_subway_hourly_ridership_2024.head()
#Looks good

,transit_timestamp,station_complex_id,station_complex,borough,ridership,latitude,longitude,Georeference,transit_date,transit_time,total_ridership_ondate,total_ridership_ondate_and_station,station_percentage_of_total_ridership
0,2024-01-01 12:00:00,157,"96 St (C,B)",Manhattan,3,40.791640,-73.964700,POINT (-73.9647 40.79164),2024-01-01,12:00:00,992908,2171,0.218651
1,2024-01-01 12:00:00,440,"116 St (2,3)",Manhattan,5,40.802097,-73.949620,POINT (-73.94962 40.802097),2024-01-01,12:00:00,992908,2796,0.281597
2,2024-01-01 12:00:00,404,28 St (6),Manhattan,36,40.743070,-73.984260,POINT (-73.98426 40.74307),2024-01-01,12:00:00,992908,5664,0.570446
3,2024-01-01 12:00:00,477,72 St (Q),Manhattan,44,40.768800,-73.958430,POINT (-73.95843 40.7688),2024-01-01,12:00:00,992908,6425,0.647089
4,2024-01-01 12:00:00,306,125 St (1),Manhattan,2,40.815582,-73.958374,POINT (-73.958374 40.815582),2024-01-01,12:00:00,992908,2159,0.217442


# Next Steps
- These two datasets will be loaded into ArcGIS Pro for further analysis so we can see how the introduction of congestion pricing affected subway ridership before and after the policy.

In [25]:
mta_subway_hourly_ridership_2025.to_csv("cleaned/Manhattan_Subway_Ridership_2025.csv", index=False)
mta_subway_hourly_ridership_2024.to_csv("cleaned/Manhattan_Subway_Ridership_2024.csv", index=False)